# Data from GDELT on Autonomus Vehicles

## Extract News Articles

**Import libraries:**

In [1]:
import gdelt
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


**Read in input data from file queried through GDELT web interface:**

In [2]:
df = pd.read_csv("../data/raw/20191209233601.19044.gkg.txt", sep='\t')

In [3]:
df

,Date,NumArticles,Counts,Themes,Locations,Persons,Organizations,ToneData,CAMEOEvents,Sources,SourceURLs
0,20180301,1,NaN,WB_1921_PRIVATE_SECTOR_DEVELOPMENT;WB_405_BUSI...,"1#China#CH#CH#35#105#CH;4#Shanghai, Shanghai, ...",robin li;lihong qin,motor corp ltd;information technology;xinhua;s...,"0.873362445414847,2.18340611353712,1.310043668...",735029480,newsweek.com,http://www.newsweek.com/self-driving-cars-fina...
1,20180301,1,NaN,TAX_FNCACT;TAX_FNCACT_STUDENT;TAX_FNCACT_STUDE...,1#United Kingdom#UK#UK#54#-4#UK,andrew deakin,cardiff university,"4.02476780185759,4.64396284829721,0.6191950464...",NaN,theengineer.co.uk,https://www.theengineer.co.uk/formula-student-...
2,20180301,1,NaN,TAX_FNCACT;TAX_FNCACT_PILOT;EPU_ECONOMY_HISTOR...,"3#Miami, Florida, United States#US#USFL#25.774...",john kwant;sam abuelsamid;sherif marakby;carlo...,google;nissan motor co;ford motor co;consumer ...,"0.812182741116751,2.23350253807107,1.421319796...",NaN,therepublic.com,http://www.therepublic.com/2018/02/28/us-ford-...
3,20180301,2,NaN,TAX_FNCACT;TAX_FNCACT_REGULATORS;PUBLIC_TRANSP...,"3#Washington, Washington, United States#US#USD...",donald trump;derek kan;barack obama;elaine chao,railroad administration;transit administration...,"0,1.34228187919463,1.34228187919463,2.68456375...","734827665,734827620,734827665,734827975,735120...",reuters.com;reuters.com,https://www.reuters.com/article/us-autos-selfd...
4,20180301,2,NaN,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_DEUTSCH;...,"4#Shanghai, Shanghai, China#CH#CH23#31.2222#12...",li chungen;huang changjiang;deutsch fran,google;china tibet online china radio internat...,"1.34328358208955,2.08955223880597,0.7462686567...",NaN,bjreview.com.cn;bjreview.com,http://www.bjreview.com.cn/./Latest_Headlines/...
...,...,...,...,...,...,...,...,...,...,...,...
4084,20180331,2,NaN,WB_845_LEGAL_AND_REGULATORY_FRAMEWORK;WB_696_P...,"3#San Francisco, California, United States#US#...",NaN,trademark office,"0,2.11640211640212,2.11640211640212,4.23280423...",NaN,daijiworld.com;canindia.com,http://daijiworld.com/news/newsDisplay.aspx?ne...
4085,20180331,1,NaN,MANMADE_DISASTER_IMPLIED;CRISISLEX_CRISISLEXRE...,"2#Arizona, United States#US#USAZ#33.7712#-111....",mark dahncke;arnaud lagandr;santa clara;danny ...,instagram;volvo;nvidia;twitter;facebook;volksw...,"-0.811123986095017,1.27462340672074,2.08574739...","743466101,743466102,743467752",fredericknewspost.com,https://www.fredericknewspost.com/public/ap/ne...
4086,20180331,1,"KILL#5##2#Arizona, United States#US#USAZ#33.77...",MANMADE_DISASTER_IMPLIED;TAX_FNCACT;TAX_FNCACT...,"2#Arizona, United States#US#USAZ#33.7712#-111....",eric risberg,stanford center;department of public health;au...,"-3.30228619813717,2.54022015241321,5.842506350...","743587503,743588241",salon.com,https://www.salon.com/2018/03/31/the-everyday-...
4087,20180331,1,NaN,MANMADE_DISASTER_IMPLIED;EPU_ECONOMY_HISTORIC;...,"2#Arizona, United States#US#USAZ#33.7712#-111....",alex davies,NaN,"-8.16326530612245,0,8.16326530612245,8.1632653...","743576470,743577475",direktbroker.de,https://www.direktbroker.de/news/international...


**Note, this data already have sentiment analysis from Google. See first number in column `ToneData`:**

In [4]:
df.ToneData[0]

'0.873362445414847,2.18340611353712,1.31004366812227,3.49344978165939,23.5807860262009,0.873362445414847'

**Define scrapper that retrieves paragraphs from news articles that are larger than 50 characters. Total size of the record is kept below 9999 characters.**

In [58]:
tot_count = 0
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
def gettext(url):
    minparagraph=50
    maxtotal=9999
    global tot_count
    tot_count += 1
    if(tot_count%10 == 0): print(tot_count, end =" ")
    try:
        result = requests.get(url, headers=headers)
        #print(result.content)
    except:
        result = requests.get("http://google1299.com/", headers=headers) # <-- get dummy record from unexisted web site
        
    c = result.content
    soup = BeautifulSoup(c)
    tempnews=[]
    count=0
    for tag in soup.findAll('p'):
        temptext=tag.getText()
        count+=len(temptext)
        if len(temptext)>minparagraph and count<maxtotal:
            tempnews.append(temptext)
            #print(temptext)
    news=" ".join(x for x in tempnews)
    
    # clean record
    news.replace("\n", " ")
    news.replace("\t", " ")
    news.replace("\r", " ")
    news.replace("_", " ")
    news = re.sub(r'\s+', ' ', news).strip()
    return news

**The scrapper accepts an URL for news article in the following form:**

In [6]:
df["SourceURLs"][0]

'http://www.newsweek.com/self-driving-cars-finally-come-china-825845'

**The scrapper returns text of a news article as string (cleaned from newlines and multiple spaces, some html tags still might sneak in):**

In [7]:
gettext(df["SourceURLs"][0])

'China issued licenses to auto makers on Thursday that allow self-driving vehicles to be road tested for the first time, the state-owned Xinhua news agency reported. Two licenses were offered to Shanghai-based SAIC Motor Corp Ltd, and the other went to electric vehicle start-up NIO, Xinhua reported. NIO confirmed it received a license. There was no immediate comment from SAIC. The licenses would allow operators to test drive the vehicles on a 5.6-km (3.5-mile) public road in Jiading District of Shanghai, Xinhua said. The licenses were issued after Robin Li, the boss of China\'s biggest search engine Baidu, tested his firm\'s driverless car on Beijing\'s roads in July, stirring controversy as there were no rules for such a test, the agency said. NIO said it had received its license from Shanghai Municipal Government. "We will now be able to further the development of our autonomous driving technologies," NIO Co-Founder and President Lihong Qin said in the statement. Read more: Driverles

**Create new column in the dataframe wich contains text of news articles:**

In [8]:
df['text']= df['SourceURLs'].apply(lambda x: gettext(x))

10 20 30 40 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2220 2230 2240 2250 2260 2270 2280 2290 2300 2310 2320 2330 2340 2350 2360 2370 2380 2390 2400 2410 2420 2430 2440 2450 2460 2470 2480 2490 2500 2510 2520 2530 2540 2550 2560 2570 2580 2590 2600 2610 2620 2630 2640 2650 2660 2670 2680 2690 2700 2710 2720 2730 2740 2750 2760 2770 2780 2790 2800 2810 2820 2830 2840 2850 2860 2870 2880 2890 2900 2910 2920 2930 2940 2950 2960 2970 2980 2990 3000 3010 3020 3030 3040 3050 3060 3070 3080 3090 3100 3110 3120 3130 3140 3150 3160 3170 3180 3190 3200 3210 3220 3230 3240 3250 3260 3270 3280 3290 3300 3310 3320 3330 3340 3350 3360 3370 3380 3390 3400 3410 3420 3430 3440 3450 3460 3470 3480 3490 3500 3510 3520 3530 3540 3550 3560 3570 3580 3590 3600 3610 3620 3630 

**Save data frame to a file (tab separated):**

In [9]:
df.to_csv("../data/processed/20191209233601.19044.gkg.txt", sep='\t')

**Check that shape of the data frame didn't accidentally change.**

In [10]:
df.shape

(4089, 12)

**Print out examples of retrieved news articles. Record `0` looks good:**

In [11]:
df['text'][0]

'China issued licenses to auto makers on Thursday that allow self-driving vehicles to be road tested for the first time, the state-owned Xinhua news agency reported. Two licenses were offered to Shanghai-based SAIC Motor Corp Ltd, and the other went to electric vehicle start-up NIO, Xinhua reported. NIO confirmed it received a license. There was no immediate comment from SAIC. The licenses would allow operators to test drive the vehicles on a 5.6-km (3.5-mile) public road in Jiading District of Shanghai, Xinhua said. The licenses were issued after Robin Li, the boss of China\'s biggest search engine Baidu, tested his firm\'s driverless car on Beijing\'s roads in July, stirring controversy as there were no rules for such a test, the agency said. NIO said it had received its license from Shanghai Municipal Government. "We will now be able to further the development of our autonomous driving technologies," NIO Co-Founder and President Lihong Qin said in the statement. Read more: Driverles

**Print out examples of retrieved news articles. Record `4` is empty, so later we will check how many such records we got and filter them out in analysis:**

In [88]:
df['text'][4]

''

**Print out examples of retrieved news articles. Record `7` looks good but note at the end some html tags got in. It might be good to clean them later.**

In [12]:
df['text'][7]

'The company is partnering with Domino\'s Pizza and others to see how consumers react. DETROIT — Ford Motor Co. is making Miami-Dade County its new test area for self-driving vehicles. The automaker and its partners – Domino’s Pizza, ride-hailing company Lyft and delivery company Postmates – are starting pilot programs to see how consumers react to autonomous and semi-autonomous vehicles. Self-driving startup and Ford partner Argo AI already has a fleet of cars in the area making the highly detailed maps that are necessary for self-driving. Ford also will establish its first-ever autonomous vehicle terminal in Miami, where it will learn how to service and deploy its test fleet. More services will likely be introduced as the partnership goes on, including Chariot, an app-based shuttle service owned by Ford. It’s all part of Ford’s effort to find viable business models for fully autonomous vehicles and get them on the road by 2021. “This is, I think, the future of any automotive company 

**Print out examples of retrieved news articles. Last record `4088` looks good:**

In [13]:
df['text'][4088]

'Shortly after an autonomous car struck and killed a pedestrian in Arizona, one of GM\'s self-driving cars was handed a ticket after police said the vehicle got too close to a pedestrian while making a right turn. The car is owned by the San Francisco-based startup, Cruise Automation, and was formally cited for failing to yield to a pedestrian in a crosswalk. Cruise says that car\'s data shows the person was far away enough from the vehicle for it to be a non-issue, and that the car did nothing wrong. According to KPIX-TV, a witness said the cop was just writing a regular ticket like normal. Data collected by Cruise shows the pedestrian was 10.8 feet away from the self-driving car as it turned onto Harrison at 14th Street on one of the car\'s normal test routes. In a statement, the company said, "Safety is our priority in testing our self-driving vehicles. California law requires the vehicle to yield the right of way to pedestrians, allowing them to proceed undisturbed and unhurried wi

## Read Data and Check for Consistency

**Read data from text file where we saved data frame with retrieved news articles. We have to make sure that when we read it in no records are corrupted.**

In [14]:
df_check0 = pd.read_csv("../data/processed/20191209233601.19044.gkg.txt", sep='\t', index_col=0)

In [15]:
df_check0.shape

(4089, 13)

In [16]:
df_check0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4089 entries, 0 to 4088
Data columns (total 13 columns):
Unnamed: 0       4089 non-null int64
Date             4089 non-null int64
NumArticles      4089 non-null int64
Counts           586 non-null object
Themes           4001 non-null object
Locations        4089 non-null object
Persons          3578 non-null object
Organizations    3771 non-null object
ToneData         4089 non-null object
CAMEOEvents      2222 non-null object
Sources          4089 non-null object
SourceURLs       4089 non-null object
text             3669 non-null object
dtypes: int64(3), object(10)
memory usage: 415.4+ KB


In [17]:
df_check0

,Unnamed: 0,Date,NumArticles,Counts,Themes,Locations,Persons,Organizations,ToneData,CAMEOEvents,Sources,SourceURLs,text
0,0,20180301,1,NaN,WB_1921_PRIVATE_SECTOR_DEVELOPMENT;WB_405_BUSI...,"1#China#CH#CH#35#105#CH;4#Shanghai, Shanghai, ...",robin li;lihong qin,motor corp ltd;information technology;xinhua;s...,"0.873362445414847,2.18340611353712,1.310043668...",735029480,newsweek.com,http://www.newsweek.com/self-driving-cars-fina...,China issued licenses to auto makers on Thursd...
1,1,20180301,1,NaN,TAX_FNCACT;TAX_FNCACT_STUDENT;TAX_FNCACT_STUDE...,1#United Kingdom#UK#UK#54#-4#UK,andrew deakin,cardiff university,"4.02476780185759,4.64396284829721,0.6191950464...",NaN,theengineer.co.uk,https://www.theengineer.co.uk/formula-student-...,Formula Student will celebrate its 20th annive...
2,2,20180301,1,NaN,TAX_FNCACT;TAX_FNCACT_PILOT;EPU_ECONOMY_HISTOR...,"3#Miami, Florida, United States#US#USFL#25.774...",john kwant;sam abuelsamid;sherif marakby;carlo...,google;nissan motor co;ford motor co;consumer ...,"0.812182741116751,2.23350253807107,1.421319796...",NaN,therepublic.com,http://www.therepublic.com/2018/02/28/us-ford-...,"Nobody covers Columbus, Indiana and the surrou..."
3,3,20180301,2,NaN,TAX_FNCACT;TAX_FNCACT_REGULATORS;PUBLIC_TRANSP...,"3#Washington, Washington, United States#US#USD...",donald trump;derek kan;barack obama;elaine chao,railroad administration;transit administration...,"0,1.34228187919463,1.34228187919463,2.68456375...","734827665,734827620,734827665,734827975,735120...",reuters.com;reuters.com,https://www.reuters.com/article/us-autos-selfd...,WASHINGTON (Reuters) - U.S. Transportation Sec...
4,4,20180301,2,NaN,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_DEUTSCH;...,"4#Shanghai, Shanghai, China#CH#CH23#31.2222#12...",li chungen;huang changjiang;deutsch fran,google;china tibet online china radio internat...,"1.34328358208955,2.08955223880597,0.7462686567...",NaN,bjreview.com.cn;bjreview.com,http://www.bjreview.com.cn/./Latest_Headlines/...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4084,4084,20180331,2,NaN,WB_845_LEGAL_AND_REGULATORY_FRAMEWORK;WB_696_P...,"3#San Francisco, California, United States#US#...",NaN,trademark office,"0,2.11640211640212,2.11640211640212,4.23280423...",NaN,daijiworld.com;canindia.com,http://daijiworld.com/news/newsDisplay.aspx?ne...,Oops! It looks like nothing was found at this ...
4085,4085,20180331,1,NaN,MANMADE_DISASTER_IMPLIED;CRISISLEX_CRISISLEXRE...,"2#Arizona, United States#US#USAZ#33.7712#-111....",mark dahncke;arnaud lagandr;santa clara;danny ...,instagram;volvo;nvidia;twitter;facebook;volksw...,"-0.811123986095017,1.27462340672074,2.08574739...","743466101,743466102,743467752",fredericknewspost.com,https://www.fredericknewspost.com/public/ap/ne...,DETROIT — Autonomous vehicles get all the head...
4086,4086,20180331,1,"KILL#5##2#Arizona, United States#US#USAZ#33.77...",MANMADE_DISASTER_IMPLIED;TAX_FNCACT;TAX_FNCACT...,"2#Arizona, United States#US#USAZ#33.7712#-111....",eric risberg,stanford center;department of public health;au...,"-3.30228619813717,2.54022015241321,5.842506350...","743587503,743588241",salon.com,https://www.salon.com/2018/03/31/the-everyday-...,This article was originally published on The C...
4087,4087,20180331,1,NaN,MANMADE_DISASTER_IMPLIED;EPU_ECONOMY_HISTORIC;...,"2#Arizona, United States#US#USAZ#33.7712#-111....",alex davies,NaN,"-8.16326530612245,0,8.16326530612245,8.1632653...","743576470,743577475",direktbroker.de,https://www.direktbroker.de/news/international...,Bitte aktivieren Sie Javascript um die Seite o...


**How many records have missing text record?**

In [45]:
df_check0['text'].head(20)

0     China issued licenses to auto makers on Thursd...
1     Formula Student will celebrate its 20th annive...
2     Nobody covers Columbus, Indiana and the surrou...
3     WASHINGTON (Reuters) - U.S. Transportation Sec...
4                                                   NaN
5     Uber launches self-driving pilot in San Franci...
6     The page you are looking for doesn't exist.Cli...
7     The company is partnering with Domino's Pizza ...
8                                                   NaN
9     Fr Arturo Sosa-the Superior General of Jesuits...
10    Huei Peng, Director MCity, University of Michi...
11    Message of ‘Reconciliation & Justice’ was deli...
12    A few passing clouds, otherwise generally clea...
13                                                  NaN
14    We can't find the page you are looking for, we...
15    by Eric David Silicon Valley has almost as man...
16                                                  NaN
17                                              

In [66]:
print(df_check0['text'][0])
print(df_check0['SourceURLs'][0])

China issued licenses to auto makers on Thursday that allow self-driving vehicles to be road tested for the first time, the state-owned Xinhua news agency reported. Two licenses were offered to Shanghai-based SAIC Motor Corp Ltd, and the other went to electric vehicle start-up NIO, Xinhua reported. NIO confirmed it received a license. There was no immediate comment from SAIC. The licenses would allow operators to test drive the vehicles on a 5.6-km (3.5-mile) public road in Jiading District of Shanghai, Xinhua said. The licenses were issued after Robin Li, the boss of China's biggest search engine Baidu, tested his firm's driverless car on Beijing's roads in July, stirring controversy as there were no rules for such a test, the agency said. NIO said it had received its license from Shanghai Municipal Government. "We will now be able to further the development of our autonomous driving technologies," NIO Co-Founder and President Lihong Qin said in the statement. Read more: Driverless Ta

In [67]:
type(df_check0['text'][0])

str

In [44]:
print(df_check0['text'][4])
print(df_check0['SourceURLs'][4])

nan
http://www.bjreview.com.cn/./Latest_Headlines/201803/t20180301_800118854.html<UDIV>http://www.bjreview.com/./Latest_Headlines/201803/t20180301_800118854.html


In [65]:
type(df_check0['text'][4])

float

**Interestingly, if record is missing it is treated as float, so we have to use `.astype(str)` to convert it to string.**

In [46]:
print(df_check0['text'][8])
print(df_check0['SourceURLs'][8])

nan
https://factsweek.com/180611/self-driving-cars-global-market-analysis-by-trends-opportunities-challenges-revenue-forecast/


In [47]:
print(df_check0['text'][13])
print(df_check0['SourceURLs'][13])

nan
http://www.nbc-2.com/story/37612804/ford-and-miami-to-form-test-bed-for-self-driving-cars<UDIV>http://www.khq.com/story/37612804/ford-and-miami-to-form-test-bed-for-self-driving-cars<UDIV>http://www.wect.com/story/37612805/ford-and-miami-to-form-test-bed-for-self-driving-cars<UDIV>http://www.kztv10.com/story/37612804/ford-and-miami-to-form-test-bed-for-self-driving-cars<UDIV>http://www.abc-7.com/story/37612804/ford-and-miami-to-form-test-bed-for-self-driving-cars


In [49]:
print(df_check0['text'][16])
print(df_check0['SourceURLs'][16])

nan
http://iphonefresh.com/2018/03/01/california-to-allow-testing-self-driving-cars-with-no-driver.html


In [50]:
print(df_check0['text'][17])
print(df_check0['SourceURLs'][17])

nan
http://pimediaonline.co.uk/featuresinvestigations/should-our-self-driving-cars-be-utilitarian/


In [72]:
df_check0[ df_check0['text'].astype(str) == 'nan' ]

,Unnamed: 0,Date,NumArticles,Counts,Themes,Locations,Persons,Organizations,ToneData,CAMEOEvents,Sources,SourceURLs,text
4,4,20180301,2,NaN,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_DEUTSCH;...,"4#Shanghai, Shanghai, China#CH#CH23#31.2222#12...",li chungen;huang changjiang;deutsch fran,google;china tibet online china radio internat...,"1.34328358208955,2.08955223880597,0.7462686567...",NaN,bjreview.com.cn;bjreview.com,http://www.bjreview.com.cn/./Latest_Headlines/...,NaN
8,8,20180301,1,NaN,NaN,1#China#CH#CH#35#105#CH;1#Japan#JA#JA#36#138#J...,mercedes-benz audi,sales price analysis company segment;technolog...,"2.33812949640288,3.3273381294964,0.98920863309...",NaN,factsweek.com,https://factsweek.com/180611/self-driving-cars...,NaN
13,13,20180301,5,NaN,TAX_FNCACT;TAX_FNCACT_PILOT;EPU_ECONOMY_HISTOR...,"3#Miami, Florida, United States#US#USFL#25.774...",john kwant;sam abuelsamid;sherif marakby;carlo...,google;nissan motor co;ford motor co;associate...,"0.660611065235343,1.81668042939719,1.156069364...",NaN,nbc-2.com;khq.com;wect.com;kztv10.com;abc-7.com,http://www.nbc-2.com/story/37612804/ford-and-m...,NaN
16,16,20180301,1,NaN,TAX_FNCACT;TAX_FNCACT_DRIVER;ECON_STOCKMARKET;...,"2#California, United States#US#USCA#36.17#-119...",eli lilly,fdx advisors inc;ocwen financial corp,"3.38983050847458,3.38983050847458,0,3.38983050...",NaN,iphonefresh.com,http://iphonefresh.com/2018/03/01/california-t...,NaN
17,17,20180301,1,"KILL#5##3#San Francisco, California, United St...",WB_2024_ANTI_CORRUPTION_AUTHORITIES;WB_696_PUB...,"1#Germany#GM#GM#51.5#10.5#GM;3#San Francisco, ...",john zimmer;philippa foot,science technology should our self;massachuset...,"-2.78044103547459,2.4928092042186,5.2732502396...",NaN,pimediaonline.co.uk,http://pimediaonline.co.uk/featuresinvestigati...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,4043,20180330,1,"KILL#3##2#Oklahoma, United States#US#USOK#35.5...",KILL;TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_HUMAN;W...,"1#Japan#JA#JA#36#138#JA;2#Alabama, United Stat...",shoko asahara;aum shinrikyo;michael davis jr;d...,peoples intermediate court;walmart;sacramento ...,"-5.68129330254042,3.09468822170901,8.775981524...","743329715,743329795,743329836,743193485,743329...",blogspot.com,https://deathpenaltynews.blogspot.com/2018/03/...,NaN
4051,4051,20180331,1,KILL#21##1#Georgia#GG#GG#41.999981#43.499905#G...,SOVEREIGNTY;TAX_ETHNICITY;TAX_ETHNICITY_GEORGI...,"1#Latvia#LG#LG#57#25#LG;4#Moscow, Moskva, Russ...",zviad gamsakhurdia;pamela anderson;zaza soprom...,sustainable development dimitri kumsishvili;in...,"-2.94225818315557,2.13313718278779,5.075395365...",NaN,georgianjournal.ge,https://www.georgianjournal.ge/society/34375-g...,NaN
4052,4052,20180331,1,NaN,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_OREGON;L...,"2#Arizona, United States#US#USAZ#33.7712#-111....",sid leiken;andre dick,department of transportation;autonomous vehicl...,"0.533333333333333,2.13333333333333,1.6,3.73333...","743464162,743477105",nbc16.com,http://nbc16.com/news/local/the-future-of-self...,NaN
4068,4068,20180331,1,NaN,TAX_FNCACT;TAX_FNCACT_WOMAN;KILL;CRISISLEX_T03...,"3#Phoenix, Arizona, United States#US#USAZ#33.4...",elaine herzberg;patrick mcgroder,nvidia corp;toyota motor corp;uber technologie...,"-4.90463215258856,0.272479564032698,5.17711171...",NaN,malaysiandigest.com,http://malaysiandigest.com/technology/730658-m...,NaN


In [86]:
df_check0[ df_check0['text'].astype(str) == 'nan' ].shape

(420, 13)

## Check Data and Re-Download Failed Articles

In [27]:
df_check1 = pd.read_csv("../data/processed/20191209233601.19044.gkg.txt", sep='\t', index_col=0)

In [28]:
df_check1.shape

(4089, 12)

In [29]:
df_check1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4089 entries, 0 to 4088
Data columns (total 12 columns):
Date             4089 non-null int64
NumArticles      4089 non-null int64
Counts           586 non-null object
Themes           4001 non-null object
Locations        4089 non-null object
Persons          3578 non-null object
Organizations    3771 non-null object
ToneData         4089 non-null object
CAMEOEvents      2222 non-null object
Sources          4089 non-null object
SourceURLs       4089 non-null object
text             3669 non-null object
dtypes: int64(2), object(10)
memory usage: 415.3+ KB


In [21]:
df_check1['text'][7]

'The company is partnering with Domino\'s Pizza and others to see how consumers react. DETROIT — Ford Motor Co. is making Miami-Dade County its new test area for self-driving vehicles. The automaker and its partners – Domino’s Pizza, ride-hailing company Lyft and delivery company Postmates – are starting pilot programs to see how consumers react to autonomous and semi-autonomous vehicles. Self-driving startup and Ford partner Argo AI already has a fleet of cars in the area making the highly detailed maps that are necessary for self-driving. Ford also will establish its first-ever autonomous vehicle terminal in Miami, where it will learn how to service and deploy its test fleet. More services will likely be introduced as the partnership goes on, including Chariot, an app-based shuttle service owned by Ford. It’s all part of Ford’s effort to find viable business models for fully autonomous vehicles and get them on the road by 2021. “This is, I think, the future of any automotive company 

In [22]:
df_check1[ df_check1['text'].astype(str).str.contains('Authentication is required to access the Internet') ].shape

(35, 13)

In [23]:
df_check1_re = df_check1[ df_check1['text'].astype(str).str.contains('Authentication is required to access the Internet') ]['SourceURLs']

In [24]:
df_check1_re.shape

(35,)

In [25]:
df_check1_re['text'] = df_check1[ df_check1['text'].astype(str).str.contains('Authentication is required to access the Internet') ]['SourceURLs'].apply(lambda x: gettext(x))

4100 4110 4120 

/opt/anaconda3/envs/py37_NLP/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/envs/py37_NLP/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_expr = (await self._async_exec(code_obj, self.user_ns))


In [26]:
df_check1_re['text']

849                                                      
850                                                      
851                                                      
852                                                      
853     Wards Auto is part of the Informa Tech Divisio...
854                                                      
946                                                      
947                                                      
948                                                      
949     There was a problem serving your request. Plea...
950                                                      
951     It seems we can’t find what you’re looking for...
952                                                      
953                                                      
954                                                      
955                                                      
956                                                      
1499    A self

In [73]:
df_check1_re['text'][853]

'Wards Auto is part of the Informa Tech Division of Informa PLC This site is operated by a business or businesses owned by Informa PLC and all copyright resides with them. Informa PLC\'s registered office is 5 Howick Place, London SW1P 1WG. Registered in England and Wales. Number 8860726. BERLIN, March 15 (Reuters) - The Chinese government is evaluating a German law that lays the groundwork for use of self-driving cars, and will likely adopt some of the German rules, a legal expert said on Thursday. Eric Hilgendorf, who teaches law at the University of Wuerzburg and served on the German government\'s ethics commission on automated driving, said he had been invited to China in the coming weeks to present information about the law. Hilgendorf, who will give three lectures at the Renmin University of China Law School and hold high-level meetings about the new legislation, said China had adopted a great deal of German criminal and civil law, albeit with some tweaks, and he expected similar

In [75]:
df_check1['text'][853]

'Authentication is required to access the Internet using this system. A valid user ID and password must be entered when prompted. If you have questions, please contact GM Online Service Desk and provide the codes shown below. Date: Thu, 12 Dec 2019 06:47:55 GMT Username: Source IP: 10.45.142.170 URL: GET http://wardsauto.com/china-may-adopt-some-germanys-law-self-driving-cars-expert Category: Reason: UNKNOWN Notification: PROXY_AUTH_REQUIRED'

In [77]:
df_check1['text'].iloc[853]

'Authentication is required to access the Internet using this system. A valid user ID and password must be entered when prompted. If you have questions, please contact GM Online Service Desk and provide the codes shown below. Date: Thu, 12 Dec 2019 06:47:55 GMT Username: Source IP: 10.45.142.170 URL: GET http://wardsauto.com/china-may-adopt-some-germanys-law-self-driving-cars-expert Category: Reason: UNKNOWN Notification: PROXY_AUTH_REQUIRED'

In [83]:
for i,record in df_check1_re['text'].items():
    #print(i, record)
    df_check1['text'].iloc[i] = record

In [84]:
df_check1['text'][853]

'Wards Auto is part of the Informa Tech Division of Informa PLC This site is operated by a business or businesses owned by Informa PLC and all copyright resides with them. Informa PLC\'s registered office is 5 Howick Place, London SW1P 1WG. Registered in England and Wales. Number 8860726. BERLIN, March 15 (Reuters) - The Chinese government is evaluating a German law that lays the groundwork for use of self-driving cars, and will likely adopt some of the German rules, a legal expert said on Thursday. Eric Hilgendorf, who teaches law at the University of Wuerzburg and served on the German government\'s ethics commission on automated driving, said he had been invited to China in the coming weeks to present information about the law. Hilgendorf, who will give three lectures at the Renmin University of China Law School and hold high-level meetings about the new legislation, said China had adopted a great deal of German criminal and civil law, albeit with some tweaks, and he expected similar

In [85]:
df_check1[ df_check1['text'].astype(str).str.contains('Authentication is required to access the Internet') ].shape

(0, 12)

In [87]:
df_check1.to_csv("../data/processed/20191209233601.19044.gkg.txt", sep='\t')